In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
import numpy as np
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from sklearn.model_selection import cross_val_score

In [3]:
training = pd.read_csv('https://raw.githubusercontent.com/vsharathchandra/AI-Healthcare-chatbot/master/Training.csv')
testing  = pd.read_csv('https://raw.githubusercontent.com/vsharathchandra/AI-Healthcare-chatbot/master/Testing.csv')

In [4]:
cols = training.columns
cols = cols[:-1]
x    = training[cols]
y = training['prognosis']
y1 = y

cols

In [5]:
reduced_data = training.groupby(training['prognosis']).max()
#reduced_data

In [6]:
#mapping strings to numbers
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)


In [7]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=42)
testx    = testing[cols]
testy    = testing['prognosis'] 
testy   = le.transform(testy)

In [8]:
clf1 = DecisionTreeClassifier()
clf = clf1.fit(x_train,y_train)
#print(clf.score(x_train,y_train))
#print("cross result==========")
scores = cross_val_score(clf,x_test,y_test,cv=3)
#print(scores)
#print(scores.mean())
#print(cross_val_score(clf,x_test,y_test,cv=3).mean())

In [9]:
importance = clf.feature_importances_
indices = np.argsort(importance)[::-1]
features = cols

indices

In [10]:
Symptom_list = []
for f in range(132):
    print("Symptom %d -  %s" %(f+1,features[indices[f]]))
    Symptom_list.append(features[indices[f]])
print('\n')

Symptom 1 -  muscle_pain
Symptom 2 -  loss_of_appetite
Symptom 3 -  throat_irritation
Symptom 4 -  internal_itching
Symptom 5 -  hip_joint_pain
Symptom 6 -  increased_appetite
Symptom 7 -  silver_like_dusting
Symptom 8 -  enlarged_thyroid
Symptom 9 -  blood_in_sputum
Symptom 10 -  yellow_crust_ooze
Symptom 11 -  nodal_skin_eruptions
Symptom 12 -  unsteadiness
Symptom 13 -  abnormal_menstruation
Symptom 14 -  weight_loss
Symptom 15 -  nausea
Symptom 16 -  fluid_overload.1
Symptom 17 -  prominent_veins_on_calf
Symptom 18 -  depression
Symptom 19 -  lack_of_concentration
Symptom 20 -  mild_fever
Symptom 21 -  neck_pain
Symptom 22 -  altered_sensorium
Symptom 23 -  slurred_speech
Symptom 24 -  movement_stiffness
Symptom 25 -  shivering
Symptom 26 -  belly_pain
Symptom 27 -  continuous_feel_of_urine
Symptom 28 -  itching
Symptom 29 -  rusty_sputum
Symptom 30 -  receiving_unsterile_injections
Symptom 31 -  patches_in_throat
Symptom 32 -  dark_urine
Symptom 33 -  family_history
Symptom 34 -  

In [11]:
#User_Introduction
bot_format = "TANU : {bot} "
user_format = "{user} :"
n = ""

In [12]:
#Symptoms_By_User
sym = []
user_symptom = []


In [13]:
def disease_input_by_user():
    sym.clear()
    user_symptom.clear()
    print(bot_format.format(bot = "Please give your major symptoms you feeling which help me to predict Disease."))
    print(bot_format.format(bot = "NO to finish :- "))
    txt = "more"
    i =1
    while 1:
        if i!= 1:
            print(bot_format.format(bot = "MORE symptoms ?"))
        print(user_format.format(user = n), end = " ")
        symptom_feel = input()
        if symptom_feel.lower() == 'no':
            break
        if symptom_feel not in Symptom_list:
            print(bot_format.format(bot = "Please Enter Valid Symptom -"))
        else:
            sym.append(symptom_feel.lower())
        i = i+1
    for f in range(132):
        if(features[indices[f]] in sym):
            user_symptom.append(indices[f])

In [14]:
def print_disease(node):
    #print(node[0])
    node = node[0]
    #print(len(node))
    val  = node.nonzero() 
    #print(val)
    disease = le.inverse_transform(val[0])
    return disease

In [15]:
def tree_for_code():
    disease_input_by_user()
    tree_ = clf.tree_
    feature_name = [
    cols[i] if i != _tree.TREE_UNDEFINED else "undefined!" 
        for i in tree_.feature
    ]
    symptoms_present = []
    node = 0
    depth = 1
    status = 0
    def check_user(node,depth):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            a = np.array(user_symptom)
            if (tree_.feature[node] in a) :
                val = 1
            else:
                txt = "Please reply Yes or No for the following symptoms -"
                print(bot_format.format(bot = txt))
                print(bot_format.format(bot = name + "?"))
                print(user_format.format(user = n),end = " ")
                ans = input()
                ans = ans.lower()
            if ans == 'yes':
                val = 1
            else:
                val = 0
            if  val <= threshold:
                check_user(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                check_user(tree_.children_right[node], depth + 1)
        else:
            #print(tree_.value,len(tree_.value))
            present_disease = print_disease(tree_.value[node])
            print( bot_format.format(bot = ("You may have " +  present_disease)))
            red_cols = reduced_data.columns 
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            print(bot_format.format(bot = ("symptoms present  " + str(list(symptoms_present)))))
            print(bot_format.format(bot = ("symptoms given "  +  str(list(symptoms_given)) ) )) 
            confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
            #print(bot_format.format(bot = ("confidence level is " + str(confidence_level))))
                    
    for user_recent in user_symptom:
        if(status == 1):
            break
        user_importance = importance[user_recent]
        while 1:
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                g = importance[tree_.feature[node]]
                m = user_importance
                if importance[tree_.feature[node]] == user_importance:
                    name = feature_name[node]
                    symptoms_present.append(name)
                    check_user(tree_.children_right[node],depth+1)
                    status = 1
                    break
                node = node+1
                depth = depth+1
            else:
                break
                        
    if status == 0:
        print(bot_format.format(bot = "Sorry for Inconvience ! But unable to identify disease by given Symptoms."))
        print(bot_format.format(bot = "Please contact to Doctor"))
            

In [16]:
%store -r name_user
n = name_user
tree_for_code()

TANU : Please give your major symptoms you feeling which help me to predict Disease. 
TANU : NO to finish :-  
Nandini : no
TANU : Sorry for Inconvience ! But unable to identify disease by given Symptoms. 
TANU : Please contact to Doctor 
